In [2]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.models as models
from tqdm.notebook import tqdm
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import make_grid

import os
import re
import requests

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [11]:
data_dir = 'archive/Cars Detection/train/images'
print(os.listdir(data_dir))
print(len(os.listdir(data_dir)))

['005980b3a5f93085_jpg.rf.2QZCXGReqcBORWQJKQfO.jpg', '005980b3a5f93085_jpg.rf.68f1ed44dbca53b707a1d3d82d4b7d9a.jpg', '00aaf0a0a9ee7e71_jpg.rf.808b1e59067887493dffad63561c2a9d.jpg', '00aaf0a0a9ee7e71_jpg.rf.KBCzX2HtZviBuQY9JDTQ.jpg', '00e0422e47cdbeb7_jpg.rf.a8b6f2ef17a14eb8b9228d26366a124d.jpg', '00e0422e47cdbeb7_jpg.rf.yHmAn1jB2oy8XzzQPBzc.jpg', '00e13aca7d2f904a_jpg.rf.52342966f5a37167f8cfd7ea3f4dacef.jpg', '00e13aca7d2f904a_jpg.rf.eQwpgo07vRFeI7SgvXy6.jpg', '00e2d9121adc0c20_jpg.rf.50fa25618ccc9f0ebf59f2ffe64cf298.jpg', '00e2d9121adc0c20_jpg.rf.rPP5CqB9mOv9wnRTTKR3.jpg', '011a8a0a921c2376_jpg.rf.064272f1b667f68b4340d240244ff7e8.jpg', '011a8a0a921c2376_jpg.rf.tflPpvQDTaDj9gH1xuDJ.jpg', '0133391d627e6df4_jpg.rf.5fb6cc71948fdee8e2e7cbc9bbc5aeb1.jpg', '0133391d627e6df4_jpg.rf.xszyOWdyIGCiI2HmiGNr.jpg', '013476982d77e380_jpg.rf.3abcf2c4c484148e9e88d3c8c99bc236.jpg', '013476982d77e380_jpg.rf.E6oudaF78aSJULyWaSIS.jpg', '0158307c4ad94b38_jpg.rf.6VgRLvEWkhSwfdBdY9Fi.jpg', '0158307c4ad94b38_j

In [12]:
#Making a list that contains the paths of each image
def get_path_names(dir):
  images = []
  for path, subdirs, files in os.walk(data_dir):  
    for name in files:
      #print(os.path.join(path, name)) 
      images.append(os.path.join(path, name))
  return images

In [13]:
classes = ['Ambulance', 'Bus', 'Car', 'Motorcycle', 'Truck']
len(classes)

5

In [14]:
def encode_label(label, classes_list = classes): #encoding the classes into a tensor of shape (11) with 0 and 1s.
    target = torch.zeros(11)
    for l in label:
      idx = classes_list.index(l)
      target[idx] = 1
    return target


def decode_target(target, threshold=0.5): #decoding the prediction tensors of 0s and 1s into text form
    result = []
    for i, x in enumerate(target):
        if (x >= threshold):
          result.append(classes[i])     
    return ' '.join(result)

In [15]:
# A class to create a Custom Dataset that will load images and encode the labels of those images from their folder names
class myDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.root_dir = root_dir
        self.images = get_path_names(root_dir)              
      

    def __len__(self):
        return len(self.images)       

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = Image.open(img_path)
       
        if self.transform:
            img = self.transform(img)    
            
        label = re.findall(r'\w+\_\w+', img_path)[0].split('_') 
        
        return img, encode_label(label)  

In [16]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # mean and std values of the Imagenet Dataset so that pretrained models could also be used


#setting a set of transformations to transform the images 
transform= T.Compose([T.Resize(128),
                      T.RandomCrop(128),
                      T.RandomHorizontalFlip(),
                      T.RandomRotation(2),
                      T.ToTensor(),
                      T.Normalize(*imagenet_stats)])

In [17]:
#Creating a dataset that loads images from the specified directory, encode their labels and transforming them into tensors.
dataset = myDataset(data_dir, transform = transform)
len(dataset)

878

In [18]:
dataset[1]

ValueError: '005980b3a5f93085' is not in list